# Part 1: Streams

Data: MovieLens 1M

Environment: Colab

Description: Read the user-item data (ratings.dat file) as a stream, line by line. You can't store the lines in memory or on disk Then, create a random sample that gets approximately 10% of the movies, and count the approximate number of different users on that stream.

Technologies: Python, only "random" library 


In [83]:
import random

In [84]:
SEED = 42
ESTIMATORS = 128
PRIME = 2147483647
HASH_RANGE = 2**32
CORRECTION = 0.77351
SAMPLE_RATE = 0.1
RATINGS_PATH = 'data/ml-1m/ratings.dat'

In [85]:
def count_trailing_zeros(number):
    if number == 0:
        return 0
    count = 0
    while (number & 1) == 0:
        count += 1
        number >>= 1
    return count

In [86]:
def is_movie_sampled(movie_id):
    random.seed(movie_id)
    return random.random() < SAMPLE_RATE

In [87]:
def generate_hash_parameters():
    parameters = []
    for _ in range(ESTIMATORS):
        multiplier = random.randint(1, PRIME - 1)
        offset = random.randint(0, PRIME - 1)
        parameters.append((multiplier, offset))
    return parameters

In [88]:
def compute_hash(value, multiplier, offset):
    hashed = (multiplier * value + offset) % PRIME
    hashed = hashed % HASH_RANGE
    return hashed

In [89]:
def update_estimator(identifier, parameters, max_zeros):
    for index in range(len(parameters)):
        multiplier, offset = parameters[index]
        hashed = compute_hash(identifier, multiplier, offset)
        zeros = count_trailing_zeros(hashed)
        if zeros > max_zeros[index]:
            max_zeros[index] = zeros

In [90]:
def estimate_distinct(max_zeros):
    estimates = []
    for zeros in max_zeros:
        estimates.append(2 ** zeros)
    estimates.sort()
    
    middle = len(estimates) // 2
    left = estimates[middle - 1]
    right = estimates[middle]
    median = (left + right) / 2
    
    return int(median * CORRECTION)

In [91]:
def process_stream(user_parameters, movie_parameters):
    user_max_zeros = [0] * ESTIMATORS
    movie_max_zeros = [0] * ESTIMATORS
    sample_count = 0
    
    file = open(RATINGS_PATH, 'r', encoding='latin-1')
    for line in file:
        line = line.strip()
        parts = line.split('::')
        if len(parts) != 4:
            continue
        
        user_id = int(parts[0])
        movie_id = int(parts[1])
        
        if is_movie_sampled(movie_id):
            sample_count += 1
            update_estimator(user_id, user_parameters, user_max_zeros)
            update_estimator(movie_id, movie_parameters, movie_max_zeros)
    file.close()
    
    return sample_count, user_max_zeros, movie_max_zeros

## Sampling

In [92]:
random.seed(SEED)

user_parameters = generate_hash_parameters()
movie_parameters = generate_hash_parameters()

sample_count, user_max_zeros, movie_max_zeros = process_stream(user_parameters, movie_parameters)

distinct_users = estimate_distinct(user_max_zeros)
distinct_movies = estimate_distinct(movie_max_zeros)

print("Estimated distinct movies:", distinct_movies)
print("Estimated distinct users:", distinct_users)
print("Sampled ratings count:", sample_count)

Estimated distinct movies: 396
Estimated distinct users: 6336
Sampled ratings count: 118625


## Evaluation

In [93]:
actual_users = set()
actual_movies = set()

file = open(RATINGS_PATH, 'r', encoding='latin-1')
for line in file:
    line = line.strip()
    parts = line.split('::')
    if len(parts) != 4:
        continue
    
    user_id = int(parts[0])
    movie_id = int(parts[1])
    
    if is_movie_sampled(movie_id):
        actual_users.add(user_id)
        actual_movies.add(movie_id)
file.close()

print("Actual distinct movies:", len(actual_movies))
print("Actual distinct users:", len(actual_users))
print("Movies error:", round(abs(distinct_movies - len(actual_movies)) / len(actual_movies) * 100, 1), "%")
print("Users error:", round(abs(distinct_users - len(actual_users)) / len(actual_users) * 100, 1), "%")

Actual distinct movies: 385
Actual distinct users: 5980
Movies error: 2.9 %
Users error: 6.0 %
